# Sales forecasting using RNN

In [2]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import pandas as pd
from IPython.display import Image

In the following notebook, I use RNN on a bigdata dataframe of sales in Russia. Where we have information about 60 stores and 21807 between January 2013 and October 2015, and we want to predict the quantity of each product that will be sold in each store in November 2015.
 
![title](OriginalData.png)
 
For this purpose I used RNN which is a neural network composed of recurrent cells.
The following image represent very well these cells: 
 
![title](RecurrentCell.png)
(Image taken from: 
Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, Aurélien,G)

In the image it is only one cell expanded in time. So the output of the cell in timestep t is one of the inputs (the original input is the other input) of the cell in the timestep t+1. But as we do not want to use only one cell but more than one, we used instead a layer of recurrent cells:

![title](RecurrentLayer.png)
(Image taken from: 
Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, Aurélien,G)

But in a bigdata problem, with many information of a relative short period of time the regular recurrent layer do not allow us to find patterns, for this reason I had to use LSTM, which are cells that not only receive the output of the previous cell but also save some information about cells before t. This cells are represented in the following picture:

![title](LSTMcell.png)
(Image taken from: 
Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, Aurélien,G)

The following code print the predicted values for the problem but also save them in a file named Prediction.csv.
The results are bounded using the function clip because it is known that the results are suppossed to be between 0 and 20.

In [1]:
sales = pd.read_csv("sales_train_v2.csv")

sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')
sales['month'] = sales['date'].dt.month
sales['year'] = sales['date'].dt.year
sales = sales.drop(['date','item_price'], axis=1)

sales = sales.groupby([col for col in sales.columns if col not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
sales = sales.rename(columns={'item_cnt_day':'item_cnt_month'})

np.zeros([len(set(sales.shop_id)),len(set(sales.item_id))])

[[len(set(sales.shop_id)),len(set(sales.item_id))]]

data = [[np.zeros(max(set(sales.item_id))+1) for i in range(max(set(sales.date_block_num))+1)] for j in range(max(set(sales.shop_id))+1)]

for shop in set(sales.shop_id):
    for date in set(sales.date_block_num[sales.shop_id==shop]):
        for item in set(sales.item_id[sales.shop_id==shop][sales.date_block_num==date]):
            data[shop][date][item]=float(sales[sales.shop_id==shop][sales.date_block_num==date][sales.item_id==item].item_cnt_month)
#Organizacion de los datos en un arreglo (Para agilizar se entrega el arreglo como un archivo de numpy)

dataNP = np.load("data.npy")

#Creacion del modelo

tf.reset_default_graph()

n_steps = 12
n_inputs = 60
n_neurons = 250
n_outputs = 60
n_layers = 3

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None,n_inputs])

#Celulas LSTM
lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
#Multicapas
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
top_layer_h_state = states[-1][1]
#Capa densa
y_pred = tf.layers.dense(top_layer_h_state, n_outputs)

loss = tf.reduce_mean(tf.losses.mean_squared_error(y,y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
   
init = tf.global_variables_initializer()


X_train = dataNP
    
#Definicion del x para el test
X_test = dataNP[:,34-n_steps:,:]
x_test = []
for item in range(max(set(sales.item_id))+1):
    x_test.append(X_test[:,:,item].T)
    
n_epochs = 1
batch_size = 1024

L = 22170
n_batches = L//batch_size


with tf.Session() as sess:
    init.run()
    for i in range(len(dataNP[0,:,0])-n_steps):
        x_train = []
	#Reorganizacion del x de entrenamiento para que quede de la forma de X, igual con y_train
        for item in set(sales.item_id):
            x_train.append(X_train[:,i:i+n_steps,item].T)
        x_train = np.array(x_train)
        y_train = dataNP[:,i+n_steps,:].T
        for epoch in range(n_epochs):
		#El batch es un lote con items con los cuales se entrenan las celulas
            	indices=np.arange(len(x_train))
            	for batch in range(n_batches):
                	X_batch=x_train[indices[batch*batch_size:(batch+1)*batch_size]]
                	y_batch=y_train[indices[batch*batch_size:(batch+1)*batch_size]]
                	sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    val = y_pred.eval(feed_dict={X: x_test})
    print(val)

#Grabar los datos

test = pd.read_csv("test.csv")

Prediction = np.zeros(len(test))

for i in range(len(test)):
    shop = test[test.ID==i].shop_id
    item = test[test.ID==i].item_id
    Prediction[i] = val[item,shop]
    
#Ajuste de los datos entre 0 y 20 y guardar en formato submission
test["item_cnt_month"] = Prediction.clip(0,20)
test = test.drop(['shop_id','item_id'], axis=1)
test.to_csv('Prediction.csv', index=False)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.la